In [14]:
import pickle
import os
import numpy as np
import ase.io
from ase.constraints import FixAtoms
from ase.build import add_adsorbate, molecule, surface
from pymatgen.ext.matproj import MPRester
from pymatgen.core.surface import generate_all_slabs, SlabGenerator
from pymatgen.io.ase import AseAtomsAdaptor
from ocpmodels.common.relaxation.ase_utils import OCPCalculator
import json
import ray
import pandas as pd
from transformers import AutoTokenizer, AutoModel, pipeline

In [15]:
config_yml_path = "ocp/configs/is2re/all/dimenet_plus_plus/dpp.yml"
checkpoint_path = "dimenetpp_all.pt"

# Define the calculator
calc = OCPCalculator(config_yml=config_yml_path, checkpoint=checkpoint_path)

RuntimeError: !atom.empty() INTERNAL ASSERT FAILED at "/opt/conda/conda-bld/pytorch_1678402411778/work/aten/src/ATen/core/qualified_name.h":24, please report a bug to PyTorch. Invalid name for qualified name: '__torch__.ocpmodels.models..equiformer_v2.gaussian_rbf.gaussian'

In [13]:
from ocpmodels.models.base import BaseModel
from ocpmodels.common.utils import build_model
import torch

ImportError: cannot import name 'build_model' from 'ocpmodels.common.utils' (/host_workspace/v9/ocp/ocpmodels/common/utils.py)

In [ ]:
# 1. Load the pre-trained model

# Load the configuration
with open("ocp/configs/is2re/all/dimenet_plus_plus/dpp.yml", 'r') as stream:
    config = yaml.safe_load(stream)

model_config = config['model']
model = build_model(model_config)  # This function might need to be written or replaced based on how you usually instantiate models with your framework.

# Load checkpoint
checkpoint = torch.load("dimenetpp_all.pt")
model.load_state_dict(checkpoint['state_dict'])

# 2. Define a hook

embeddings = None

def hook_fn(module, input, output):
    global embeddings
    embeddings = output

# Register hook to the last output block
hook = model.output_blocks[-1].register_forward_hook(hook_fn)

# 3. Prepare the input data

# Assuming you have a function `generate_data` that constructs the data object for a given bulk_symbol, miller_index, and ads_symbol
data = generate_data(bulk_symbol="Ag12", miller_index=(1,1,0), ads_symbol="CH2CH2OH")

# 4. Call the model's forward method

with torch.no_grad():  # No need to track gradients
    model(data)

print(embeddings)

# Cleanup
hook.remove()